In [3]:
import json
from typing import (
    List,
    Optional,
)

import numpy as np
import requests
from langchain_community.vectorstores import FAISS
from langchain_core.embeddings import Embeddings


class JinaEmbeddings(Embeddings):
    def __init__(self, url: str, batch_size: Optional[int] = None) -> None:
        super().__init__()
        self.url = url
        self.batch_size = batch_size

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        payload = {"text": texts,
                   "batch_size": self.batch_size}
        response = requests.post(self.url, json=payload)

        if response.status_code == 200:
            return np.array(eval(response.text))
        else:
            return response.status_code

    def embed_query(self, text: str) -> List[float]:

        return self.embed_documents([text])[0]


def find_key_paths(dictionary, path=[]):
    """递归遍历字典以找到最小粒度的key路径列表"""
    key_paths = []
    for key, value in dictionary.items():
        # 构建当前路径
        current_path = path + [key]

        # 如果值仍然是字典，继续递归
        if isinstance(value, dict) and value:
            key_paths.extend(find_key_paths(value, current_path))
        else:
            # 如果值不是字典，或者是空字典，那么这个 key 路径就是最小粒度的路径
            key_paths.append(current_path)

    return key_paths


def get_value_from_path(dictionary, key_path):
    """根据key路径列表从字典中获取值"""
    for key in key_path:
        # 更新字典为下一层级
        dictionary = dictionary[key]
    return dictionary


def set_value_by_path(root_dict, key_path, value):
    """根据key路径列表在字典中设置值"""
    current_dict = root_dict
    for key in key_path[:-1]:  # 遍历路径中的 key，除了最后一个
        if key not in current_dict or not isinstance(current_dict[key], dict):
            current_dict[key] = {}  # 如果路径不存在，则创建新字典
        current_dict = current_dict[key]
    # 设置最终的 key 的值
    current_dict[key_path[-1]] = value


def document_to_dict(document):
    dictionary = {'page_content': document.page_content, 'metadata': document.metadata}
    return dictionary


faiss_index_path = './faiss_index/bid'

db = FAISS.load_local(faiss_index_path, embeddings=JinaEmbeddings(url="http://18.138.147.39:9032/api/v2/embed_onnx"),
                       allow_dangerous_deserialization=True)


faiss_index_path_split = './faiss_index/split_bid'

db_split = FAISS.load_local(faiss_index_path_split, embeddings=JinaEmbeddings(url="http://18.138.147.39:9032/api/v2/embed_onnx"),
                       allow_dangerous_deserialization=True)

In [8]:
with open('query.json', 'r', encoding='utf-8') as file:
    query_dict = json.load(file)

key_paths = find_key_paths(query_dict)

result = {}
prefix = '全文检索：'
search_type = 'similarity'
search_kwargs = {
    'k': 10,
    'score_threshold': 0,
    'fetch_k': 10,
    'lambda_mult': 0,
    'filter': {}
}
retriever = db.as_retriever(k=10)
retriever_split = db_split.as_retriever(k=10)

for key_path in key_paths[:1]:
    query = get_value_from_path(query_dict, key_path)
    if query:
        if query.startswith(prefix):
            docs = retriever_split.invoke(query[len(prefix):])
        else:
            docs = retriever.invoke(query)
        docs = [{**document_to_dict(doc), 'query': query} for doc in docs]
        print(query)
        print(docs)
        set_value_by_path(root_dict=result, key_path=key_path, value=docs)

with open('result/bid.json', 'w', encoding='utf-8') as file:
    file.write(json.dumps(result, ensure_ascii=False, indent=4))

项目名称 招标公告
[{'page_content': '1.3招标范围、计划工期和质量要求', 'metadata': {'section': ['第二章 投标人须知', '投标人须知前附表', '1．总则', '1.3招标范围、计划工期和质量要求'], 'page': 25, 'data': '1.3.1本次招标范围：见投标人须知前附表。 1.3.2本招标项目的计划工期：见投标人须知前附表。 1.3.3本招标项目的质量要求：见投标人须知前附表。'}, 'query': '项目名称 招标公告'}, {'page_content': '1.1.4招标范围', 'metadata': {'section': ['第七章 技术标准和要求', '1．总则', '1.1.4招标范围'], 'page': 328, 'data': '本项目各标段招标范围为：光伏区、35kV集电线路、220kV升压站的勘察设计，以及与项目有关的全部设备（不含二次监控系统以及集控子站）；光伏组件、逆变器及支架按暂估价计入）和材料的采购供应、建筑及安装工程施工、项目管理、试验及检查测试、系统调试、试运行、消缺、培训、验收（含各项专题、阶段验收、竣工验收等验收）、移交生产、性能质量保证、工程质量保修期限的服务，质保期内所有备品备件、专用工具采购供应以及相关的技术资料整理提供服务，同时也包括办理建设手续、用地手续、质量监检、并网手续、调度手续及供电手续、征地协调，并承担全部相关费用。具体招标范围包括（包括但不限于）： （1）勘察设计范围为建设功能完整的光伏电站，包括但不限于光伏区、35kV集电线路、220kV升压站、施工电源（并网后作为电站备用电源）的勘察工作；初步设计阶段包括但不限于光伏区1:500地形图测绘、220kV升压站1:500地形图测绘、地勘、初步设计报告(含审查)、设备技术规范书、材料清单、工程量清单等；编制安全设施设计专篇、职业卫生防护设施设计专篇、环境与水土保持设计专篇等；施工图设计阶段包括但不限于项目地勘、设计、计算书、采购设备图纸文件、竣工图编制；完成相关部门施工图审查工作；施工图技术交底、派出设计代表驻场并解决现场施工出现的与设计相关的技术问题等现场服务工作等。（2）光伏方阵区设备（光伏组件、逆变器及支架按暂估价计入）和材料采购，包括：所有光伏组件、逆变器、支架、箱变、电缆、光

In [2]:
import PyPDF2

def preprocess_pdf(pdf_path):
    text_by_page = []
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text = page.extract_text()
            text_by_page.append(text)
    return text_by_page

In [7]:
pages = preprocess_pdf('bid.pdf')

In [35]:
for i in range(100):
    print(i, extract_leading_numbers(pages[i]))

0 None
1 None
2 None
3 None
4 None
5 None
6 None
7 None
8 1
9 2
10 3
11 4
12 5
13 65
14 7
15 8
16 9
17 10
18 11
19 12
20 13
21 14
22 15
23 16
24 171
25 18
26 19
27 20
28 211
29 22
30 23
31 243
32 253
33 26
34 27
35 28
36 29
37 30
38 31
39 32
40 33
41 34
42 35
43 36
44 37
45 38
46 39
47 40
48 41
49 42
50 431
51 44
52 45
53 46
54 47
55 481
56 49
57 50
58 51
59 521
60 531
61 542
62 553
63 56
64 574
65 58
66 59
67 60
68 615
69 62
70 63
71 64
72 657
73 66
74 67
75 68
76 69
77 70
78 71
79 72
80 73
81 74
82 75
83 76
84 7716
85 78
86 79
87 80
88 81
89 82
90 83
91 84
92 85
93 86
94 8719
95 8820
96 8920
97 9021
98 91
99 92


In [23]:
import re
def extract_leading_numbers(s):
    # 使用正则表达式匹配字符串开头的数字
    match = re.match(r'\d+', s)
    if match:
        return match.group()  # 返回匹配到的数字字符串
    else:
        return None  # 如果开头没有数字，则返回None


In [81]:
def generate_regex_pattern(text):
    punctuation = r'[.,;:!?]'
    parts = re.split(f'({punctuation}|\s+|\d+|[a-zA-Z]+|[^a-zA-Z\d\s]+)', text)
    escaped_parts = [(r'\s*' if part.isspace() else re.escape(part)) for part in parts if part]
    pattern = r'\s*'.join(escaped_parts)
    return pattern


def find_text(text_by_page, search_text, start_page=1):
    start_index = max(start_page - 1, 0)
    pattern = generate_regex_pattern(search_text)
    compiled_pattern = re.compile(pattern)
    for i, page_text in enumerate(text_by_page[start_index:], start=start_index):
        print(i)
        if compiled_pattern.search(page_text):
            print('find !')
            print(i, page_text)
            page_before = extract_leading_numbers(text_by_page[i-1]) if i >1 else text_by_page[i]
            page_after = extract_leading_numbers(text_by_page[i+1]) if i < (len(text_by_page)-1) else text_by_page[i]
            page_len = min(len(page_before) if page_before else 1000000, len(page_after) if page_after else 10000000)
            return int(extract_leading_numbers(page_text)[:page_len]) if extract_leading_numbers(page_text) else None
    return None

In [75]:
search_text = "内蒙古库布齐沙漠鄂尔多斯中北部新能源基地700万千瓦光伏项目分为组团1和组团4两个组团，组团1包括5个工程标段（标段1-5），组团4包括3个工程标段（标段6-8），8个工程标段共建设12座220kV升压站，总建设规模为700万千瓦，场址均位于内蒙古自治区鄂尔多斯市达拉特旗，场址海拔约1060m~1150m，场址附近有兴巴高速、乌漫线经过，对外交通条件较便利。组团及标段划分示意见图2.1;"

In [76]:
pattern = generate_regex_pattern(search_text[:10])
compiled_pattern = re.compile(pattern)
compiled_pattern.search(pages[8])

<re.Match object; span=(9, 19), match='内蒙古库布齐沙漠鄂尔'>

In [97]:
pages[63]

'56承包人在履行合同过程中应遵守法律，并保证发包人免于承担因承包人违反法律\n而引起的任何责任。\n4.1.2依法纳税\n承包人应按有关法律规定纳税，应缴纳的税金包括在合同价格内。\n4.1.3完成各项承包工作\n承包人应按合同约定以及监理人根据第3.4款作出的指示，完成合同约定的全部\n工作，并对工作中的任何缺陷进行整改、完善和修补，使其满足合同约定的目的。除\n专用合同条款另有约定外，承包人应提供合同约定的工程设备和承包人文件，以及为\n完成合同工作所需的劳务、材料、施工设备和其他物品，并按合同约定负责临时设施\n的设计、施工、运行、维护、管理和拆除。\n4.1.4对设计、施工作业和施工方法，以及工程的完备性负责\n承包人应按合同约定的工作内容和进度要求，编制设计、施工的组织和实施计划，\n并对所有设计、施工作业和施工方法，以及全部工程的完备性和安全可靠性负责。\n4.1.5保证工程施工和人员的安全\n承包人应按第10.2款约定采取施工安全措施，确保工程及其人员、材料、设备\n和设施的安全，防止因工程施工造成的人身伤害和财产损失。\n4.1.6负责施工场地及其周边环境与生态的保护工作\n承包人应按照第10.4款约定负责施工场地及其周边环境与生态的保护工作。\n4.1.7避免施工对公众与他人的利益造成损害\n承包人在进行合同约定的各项工作时，不得侵害发包人与他人使用公用道路、水\n源、市政管网等公共设施的权利，避免对邻近的公共设施产生干扰。承包人占用或使\n用他人的施工场地，影响他人作业或生活的，应承担相应责任。\n4.1.8为他人提供方便\n承包人应按监理人的指示为他人在施工场地或附近实施与工程有关的其他各项\n工作提供可能的条件。除合同另有约定外，提供有关条件的内容和可能发生的费用，\n由监理人按第3.5款商定或确定。\n4.1.9工程的维护和照管\n工程接收证书颁发前，承包人应负责照管和维护工程。工程接收证书颁发时尚有\n部分未竣工工程的，承包人还应负责该未竣工工程的照管和维护工作，直至竣工后移\n交给发包人。'

In [88]:
find_text(pages, search_text[:10], start_page=8)

7
8
find !
8 1第一章招标公告
内蒙古库布齐沙漠鄂尔多斯中北部新能源基地700万千瓦
光伏项目工程设计施工采购招标公告
（招标编号：T231100111507）
1．招标条件
本招标项目内蒙古库布齐沙漠鄂尔多斯中北部新能源基地700万千瓦光伏
项目工程设计施工，已获批准建设，建设资金来自招标人自有资金和自筹资金，
招标人为内蒙古三峡蒙能能源有限公司，招标代理机构为三峡国际招标有限责
任公司。项目已具备招标条件，现对该项目的设计施工进行公开招标。
2．项目概况与招标范围
2.1项目概况
内蒙古库布齐沙漠鄂尔多斯中北部新能源基地700万千瓦光伏项目分为组
团1和组团4两个组团，组团1包括5个工程标段（标段1-5），组团4包括3
个工程标段（标段6-8），8个工程标段共建设12座220kV升压站，总建设规
模为700万千瓦，场址均位于内蒙古自治区鄂尔多斯市达拉特旗，场址海拔约
1060m~1150m，场址附近有兴巴高速、乌漫线经过，对外交通条件较便利。组
团及标段划分示意见图2.1;
图2.1组团及标段划分示意图
2.1.1组团1概况：
组团1位于达拉特旗县城西南侧，海拔约1060m~1150m。达拉特旗距包头
机场25公里、鄂尔多斯机场110公里、呼和浩特机场150公里。组团1场址地
势较为平坦，对外交通便利，适宜光伏场地建设。场外运输可利用较为完善的高


1